# Relatórios Metodológicos para Pesquisa em Interfaces Gráficas de Periódicos Digitalizados

Com esse notebook, você pode registrar os parâmetros e metadados de pesquisas realizadas em interfaces de busca de periódicos digitalizados. O objetivo é facilitar a replicação de pesquisas, garantir transparência metodológica, comparação de resultados e o próprio processo de análise e escrita.

## Exemplo de relatórios

Ao final da execução, você terá relatórios de pesquisa no formato txt e csv, podendo inclusive unir (concatenar) diferentes relatórios em um único arquivo.

O relatório em texto simples (`.txt`) tem o seguinte padrão:

```
--------------------------------------------------------------------------------
Dados gerais da pesquisa
Nome do repositório: Hemeroteca Digital Brasileira
URL do repositório: http://memoria.bn.br/hdb/periodico.aspx
Nome do usuário: Eric Brasil
Data: 2023-09-25
Hora: 14:51:46
--------------------------------------------------------------------------------
Parâmetros da pesquisa
Resumo da pesquisa: Pesquisa tendo Local como primeiro parâmetro. Exemplo para Hemdig(pt)
Local: RJ
Recorte temporal: 1900 - 1909
Periódico pesquisado: Todos
Termos pesquisados: "cordão carnavalesco"
--------------------------------------------------------------------------------
Resultados da pesquisa
Link da página com resultados da busca: http://memoria.bn.br/DocReader/docmulti.aspx?bib=%5Bcache%5Drosa_4756005976831.DocLstX&pasta=ano%20190&pesq=%22cord%C3%A3o%20carnavalesco%22
Número de ocorrências encontradas: 115

```

O relatório em csv (`.csv`) tem o seguinte padrão:

```
Repositório,URL,Usuário,Data da pesquisa,Resumo da pesquisa,Local,Recorte temporal,Termos pesquisados,Link da página com resultados da busca,Número de ocorrências encontrados
Hemeroteca Digital Brasileira,http://memoria.bn.br/hdb/periodico.aspx,Eric Brasil,2023-09-25_14-51-46,Pesquisa tendo Local como primeiro parâmetro. Exemplo para Hemdig(pt),RJ,1900 - 1909,"""cordão carnavalesco""",http://memoria.bn.br/DocReader/docmulti.aspx?bib=%5Bcache%5Drosa_4756005976831.DocLstX&pasta=ano%20190&pesq=%22cord%C3%A3o%20carnavalesco%22,115

```

E pode ser aberto em qualquer editor de planilhas, como o Excel, LibreOffice Calc, Google Sheets, etc.

Vejamos um exemplo de csv concatenado com dois relatórios de busca:

| Repositório | URL | Usuário | Data da pesquisa | Resumo da pesquisa | Local | Recorte temporal | Termos pesquisados | Link da página com resultados da busca | Número de ocorrências encontrados |
| --- | --- | --- | --- | --- | --- | --- | --- | --- | --- |
| Hemeroteca Digital Brasileira | http://memoria.bn.br/hdb/periodico.aspx | Eric Brasil | 2023-09-25_14-57-10 | Exemplo para Hemdig(pt) | SP | 1910 - 1919 | "cordão carnavalesco" | http://memoria.bn.br/DocReader/docmulti.aspx?bib=%5Bcache%5Ddrummond_3076505580561.DocLstX&pasta=ano%20191&pesq=%22cord%C3%A3o%20carnavalesco%22 | 43 |
| Hemeroteca Digital Brasileira | http://memoria.bn.br/hdb/periodico.aspx | Eric Brasil | 2023-09-25_14-54-30 | Pesquisa com parâmetro Local como base. Pesquisa para o Hemdig(pt) | SP | 1900 - 1909 | "cordão carnavalesco" | http://memoria.bn.br/DocReader/docmulti.aspx?bib=%5Bcache%5Drosa_5960507707285.DocLstX&pasta=ano%20190&pesq=%22cord%C3%A3o%20carnavalesco%22 | 0 |
| Hemeroteca Digital Brasileira | http://memoria.bn.br/hdb/periodico.aspx | Eric Brasil | 2023-09-25_14-59-01 | Exemplo para Hemdig(pt) | RJ | 1910 - 1919 | "cordão carnavalesco" | http://memoria.bn.br/DocReader/docmulti.aspx?bib=%5Bcache%5Ddrummond_3675006691793.DocLstX&pasta=ano%20191&pesq=%22cord%C3%A3o%20carnavalesco%22 | 278 |
| Hemeroteca Digital Brasileira | http://memoria.bn.br/hdb/periodico.aspx | Eric Brasil | 2023-09-25_14-51-46 | Pesquisa tendo Local como primeiro parâmetro. Exemplo para Hemdig(pt) | RJ | 1900 - 1909 | "cordão carnavalesco" | http://memoria.bn.br/DocReader/docmulti.aspx?bib=%5Bcache%5Drosa_4756005976831.DocLstX&pasta=ano%20190&pesq=%22cord%C3%A3o%20carnavalesco%22 | 115 |

Os exemplos podem ser vistos na pasta `reports` e o arquivo csv com os dados concatenados podem ser vistos [aqui](../../book/part3/capitulo7/reports/Hemeroteca%20Digital%20Brasileira/Hemeroteca%20Digital%20Brasileira_concatenated.csv)

## Instalar bibliotecas

A biblioteca pandas será necessária para juntar diferentes arquivos csv. Como ela não é nativa do Python, é preciso instalá-la. O jupyter notebook nos permite instalar bibliotecas diretamente no notebook. Para isso, basta executar o comando pip, precedido do sinal de porcentagem (%), seguido do nome da biblioteca que se deseja instalar.

In [ ]:
# Instalando a biblioteca pandas
%pip install pandas

## Importar bibliotecas

Aqui importamos as bibliotecas necessárias para a execução do código.

In [ ]:
# Importando bibliotecas
import csv # Biblioteca para manipulação de arquivos csv
import datetime # Biblioteca para manipulação de datas
import os # Biblioteca para manipulação do sistema operacional
import pandas as pd # Biblioteca para manipulação de dataframes
import pathlib # Biblioteca para manipulação de diretórios

:::{note}
Esta célula só precisa ser executada uma vez.
:::

## Preparação inicial: criação do diretório geral e coleta de dados sobre o repositório

Vamos criar um diretório para armazenamento dos relatórios. Em seguida coletamos as informações sobre o nome do repositório, sua URL base para pesquisa e nome do usuário que realiza a pesquisa.


In [ ]:
# Cria a pasta "reports" se ela não existir
pathlib.Path("reports").mkdir(exist_ok=True)

# Coleta nome do repositório e nome do usuário
repo_name = input("Informe o nome do repositório: ")
repo_url = input("Informe a URL do repositório: ")
user_name = input("Informe seu nome: ")

:::{note}
A célula anterior, só precisa ser executada uma vez para cada repositório. Caso você realize diversas pesquisas em um mesmo repositório, basta repetir as células abaixo para cada pesquisa.
:::

## Primeira parte: coleta de dados gerais

Essa etapa cria o arquivo de metadados da pesquisa. Esse arquivo será usado para armazenar todos os dados coletados durante a pesquisa. 

Nessa primeira etapa o arquivo é criado, e os dados sobre data, hora, nome do usuário, nome do repositório e URL base são armazenados.

:::{note}
As etapas 1, 2 e 3 devem ser executadas para cada conjunto de parâmetros de pesquisa em um mesmo repositório.
:::

In [ ]:
# Cria hora e data da execução
now = datetime.datetime.now()
search_time = now.strftime('%Y-%m-%d_%H-%M-%S')

# Cria o nome do arquivo de texto de saída
file_name = f"{repo_name}_{search_time}.txt"

# cria o caminho completo do arquivo
file_path = pathlib.Path("reports") / repo_name / file_name

# Cria a pasta com o nome do repositório se ela não existir
file_path.parent.mkdir(exist_ok=True)

# Cria o arquivo e escreve os dados gerais da pesquisa
with open(file_path, "w") as f:
    f.write("-" * 80 + "\n")
    f.write("Dados gerais da pesquisa\n")
    f.write(f"Nome do repositório: {repo_name}\n")
    f.write(f"URL do repositório: {repo_url}\n")
    f.write(f"Nome do usuário: {user_name}\n")
    f.write(f"Data: {now.strftime('%Y-%m-%d')}\n")
    f.write(f"Hora: {now.strftime('%H:%M:%S')}\n")

print(f"Arquivo {file_path} criado com sucesso!")


# Segunda parte: parâmetros da pesquisa

Nessa etapa, vamos coletar os parâmetros da pesquisa. Esses parâmetros são: local, data, palavra-chave, descrição breve, e outros.

Esses dados são incluídos no arquivo de texto criado na primeira etapa.

Também é criado um arquivo csv com os mesmos dados, para facilitar a visualização e análise dos dados.

In [ ]:
# Adiciona os parâmetros da pesquisa ao arquivo
with open(file_path, "a") as f:
    # Pede os parâmetros da pesquisa
    print("Informe os parâmetros da pesquisa, conforme solicitado.\n\
    Caso não tenha um parâmetro específico, deixe o campo vazio.\n")
    local = input("Informe o parâmetro para local: ")
    data = input("Informe o parâmetro para data: ")
    periodico = input("Informe o parâmetro para periódico: ")
    palavra_chave = input("Informe os termos utilizados na pesquisa: ")
    resumo = input("Escreva um breve resumo para esse conjunto de parâmetros: ")
    
    # Escreve os parâmetros no arquivo
    f.write("-" * 80 + "\n")
    f.write("Parâmetros da pesquisa\n")
    f.write(f"Resumo da pesquisa: {resumo}\n")
    f.write(f"Local: {local}\n")
    f.write(f"Recorte temporal: {data}\n")
    f.write(f"Periódico pesquisado: {periodico}\n")
    f.write(f"Termos pesquisados: {palavra_chave}\n")

    # Cria o arquivo CSV com os dados
    csv_file_name = file_name.replace(".txt", ".csv")
    csv_file_path = pathlib.Path("reports") / repo_name / csv_file_name
    csv_file_path.parent.mkdir(exist_ok=True)

    with open(csv_file_path, "w", newline="") as csv_file:
        writer = csv.writer(csv_file)
        writer.writerow(["Repositório", "URL", "Usuário", "Data da pesquisa", "Resumo da pesquisa", "Local", "Recorte temporal", "Termos pesquisados"])
        writer.writerow([repo_name, repo_url, user_name, search_time, resumo, local, data, palavra_chave])
print("Parâmetros da pesquisa adicionados ao arquivo!")

## Terceira parte: registro dos resultados

Aqui, coletamos os dados sobre os resultados da pesquisa. Esses dados são: link da página de resultados e número total de ocorrências.

In [ ]:
# Pede os dados sobre os resultados
result_link = input("Informe o link da página com resultados da busca: ")
result_number = input("Informe o número de ocorrências encontrados: ")

# Adiciona os dados no CSV como novas colunas ao lado dos parâmetros da pesquisa
with open(csv_file_path, "r") as csv_file:
    reader = csv.reader(csv_file)
    rows = list(reader)
    rows[0].extend(["Link da página com resultados da busca", "Número de ocorrências encontrados"])
    rows[1].extend([result_link, result_number])

# Reescreve o arquivo CSV com os novos dados
with open(csv_file_path, "w", newline="") as csv_file:
    writer = csv.writer(csv_file)
    writer.writerows(rows)

# Adiciona os dados no arquivo TXT
with open(file_path, "a") as f:
    f.write("-" * 80 + "\n")
    f.write("Resultados da pesquisa\n")
    f.write(f"Link da página com resultados da busca: {result_link}\n")
    f.write(f"Número de ocorrências encontradas: {result_number}\n")
    
print("Resultados da pesquisa adicionados ao arquivo!")    

## Quarta parte: Concatenar arquivos CSV

Nessa etapa, temos a opção de concatenar todos os arquivos csv que estão no diretório de rseultados de um mesmo periódico. Isso facilita a visualização, recuperação e análise dos dados.

:::{note}
É recomendável que essa etapa seja realizada ao final de um conjunto de pesquisas em um mesmo periódico. Assim, após reunir uma série de arquivos csv, podemos concatená-los e obter um único arquivo com todos os dados.
:::

In [ ]:
# Pergunta se o usuário deseja concatenar os arquivos CSV
concatenate_csv = input("Deseja concatenar todos os arquivos CSV existentes na pasta? (S/N) ")

# Se a resposta for sim, concatena os arquivos
if concatenate_csv.upper() == "S":
    csv_files = [f for f in os.listdir(f"reports/{repo_name}") if f.endswith(".csv")]
    if len(csv_files) > 1:
        df = pd.concat([pd.read_csv(os.path.join(f"reports/{repo_name}", f)) for f in csv_files])
        concatenated_csv_file_path = pathlib.Path("reports") / repo_name / f"{repo_name}_concatenated.csv"
        df.to_csv(concatenated_csv_file_path, index=False)
        print(f"Arquivo com todos os dados concatenados criado em {concatenated_csv_file_path}.")
    else:
        print("Não há arquivos CSV suficientes para concatenar.")
# Se a resposta for não, encerra o programa
else:
    print("Operação de concatenação cancelada pelo usuário.")